# Step 0: 環境構築とライブラリのインポート

このノートブック全体の処理を実行するために必要な,すべてのPythonライブラリをインストールし,インポートします.

### \#\# 実行内容 ⚙️

1.  **ライブラリの一括インストール**:

      * `!pip install -r requirements.txt` コマンドが,`requirements.txt`に記載されたすべてのライブラリを一度にインストールします.

2.  **ライブラリのインポート**:

      * インストールしたライブラリや,Pythonに標準で組み込まれているライブラリ（`os`, `json`など）を,このノートブックのメモリに読み込み,後続のセルで使えるようにします.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import json
import os
import re
import shutil
import subprocess
import sys

import requests
from datetime import datetime, timezone

# Step 1: リポジトリのクローン

このノートブックの最初のステップとして. SBOMの分析対象となる複数のGitリポジトリを自動的にクローンします.

### \#\# 事前準備 📝

**`url_list.txt`** という名前のファイルをこのノートブックと同じ階層に作成してください.
ファイルの中には. クローンしたいリポジトリのURLを1行に1つずつ記述します.

**`url_list.txt` の記述例:**

```
https://github.com/user1/repo1.git
https://github.com/user2/repo2.git
https://github.com/user3/another-repo.git
```

### \#\# 実行内容 ⚙️

以下のコードは. `url_list.txt` を読み込み. 各URLに対して `git clone` コマンドを実行します.

  * クローンされたリポジトリは. **`cloned_repositories`** ディレクトリ内に保存されます.
  * 既に同名のリポジトリが存在する場合は. 時間短縮のためクローン処理をスキップします.
  * クローンの進捗状況はリアルタイムで表示されます.

In [ ]:
import os
import subprocess
import sys
import shutil

# --- 設定項目 ---
# クローン対象のリポジトリURLリストファイル
url_file_path = 'url_list.txt'
# リポジトリのクローン先ディレクトリ
clone_to_directory = 'cloned_repositories'

# --- 処理の開始 ---
print(f"--- Starting: Cloning repositories ---")

# 保存先ディレクトリを作成する
os.makedirs(clone_to_directory, exist_ok=True)
print("-" * 50)

# URLリストファイルを読み込む
try:
    with open(url_file_path, 'r') as file:
        # 空行を除外してリスト化する
        urls = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print(f"❌ Error: '{url_file_path}' was not found.")
    urls = []

for repo_url in urls:
    # --- 既存リポジトリのスキップ処理 ---
    repo_name = repo_url.split('/')[-1].replace('.git', '')
    repo_path = os.path.join(clone_to_directory, repo_name)
    
    if os.path.isdir(repo_path):
        print(f"🟢 Skipping: {repo_name} (Directory already exists)")
        print("-" * 50)
        continue

    print(f"Cloning: {repo_url}")
    try:
        # git cloneプロセスを開始
        process = subprocess.Popen(
            ['git', 'clone', '--progress', repo_url],
            cwd=clone_to_directory,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8',
            errors='replace'
        )

        # 標準エラーをリアルタイムで表示
        while process.poll() is None:
            line = process.stderr.readline()
            if line:
                print(f"   {line.strip()}", end='\r')
        
        print(" " * 80, end="\r") # 進捗表示行をクリア

        if process.returncode == 0:
            print("✅ Clone successful.")

            print(f"   Searching and removing hidden directories in '{repo_name}'...")
            
            # クローンしたリポジトリ内を再帰的に探索
            for root, dirs, files in os.walk(repo_path):
                # .で始まるディレクトリをリストアップして削除
                hidden_dirs = [d for d in dirs if d.startswith('.')]
                for d_name in hidden_dirs:
                    dir_to_remove = os.path.join(root, d_name)
                    try:
                        shutil.rmtree(dir_to_remove)
                        print(f"   🗑️ Removed: {dir_to_remove}")
                    except OSError as e:
                        print(f"   ❌ Error removing {dir_to_remove}: {e}")
                
                # dirsリストから削除したものを除外し、それ以上深く探索しないようにする
                dirs[:] = [d for d in dirs if not d.startswith('.')]
            
            print("   ✅ Hidden directories removed.")

        else:
            # エラー内容を取得して表示
            stdout_err, stderr_err = process.communicate()
            error_message = stderr_err if stderr_err else stdout_err
            print(f"❌ Error cloning. Reason: {error_message.strip()}")

    except FileNotFoundError:
        print("❌ Error: 'git' command not found. Please install Git and ensure it is in your system's PATH.")
        break
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("-" * 50)

print("Clone process finished.")

# Step 2: SBOMの生成 (sbom-tool)

前のステップでクローンした各リポジトリに対して,Microsoftの **`sbom-tool`** を使用してベースとなるSBOMを生成します.このツールは,リポジトリ内の全ファイルをスキャンし,ハッシュ値を含む詳細なファイル情報を取得することに優れています.

### ## 事前準備 📝

* **`sbom-tool`** がシステムにインストールされ,PATHが通っている必要があります.
* Step 1が完了しており,**`cloned_repositories`** ディレクトリ内に分析対象のリポジトリが存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`cloned_repositories`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  **パラメータの自動取得**:
    * `git`コマンドを実行し,現在のコミットハッシュを **`packageVersion`** として取得します.
    * `.git/config`ファイルを解析し,リポジトリの所有者（OrganizationまたはUser）を **`packageSupplier`** として自動で設定します.

2.  **`sbom-tool` の実行**:
    * 取得したパラメータ（パッケージ名,バージョン,サプライヤー）を使って`sbom-tool generate`コマンドを動的に構築し,実行します.
    * これにより,各リポジトリのルートに **`_manifest`** ディレクトリが生成されます.

3.  **成果物の移動**:
    * 生成された`_manifest`ディレクトリを,後続の処理で扱いやすいように **`generated_sboms/<リポジトリ名>/source/`** ディレクトリ内に移動します.

すでに成果物が存在する場合,この処理はスキップされます.

In [ ]:
# --- 設定項目 ---
clone_to_directory = 'cloned_repositories'
sbom_output_directory = 'generated_sboms'
url_file_path = 'url_list.txt'

# --- 処理の開始 ---
print(f"--- Starting: Generating SBOMs with sbom-tool")
print("-" * 50)

original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    # --- url_list.txtからリポジトリ名と提供者のマップを作成 ---
    repo_to_supplier_map = {}
    try:
        with open(url_file_path, 'r') as file:
            urls = [line.strip() for line in file.readlines() if line.strip()]
            for url in urls:
                # URLから所有者とリポジトリ名を抽出
                match = re.search(r"github\.com/([^/]+)/([^/.]+)", url)
                if match:
                    owner, repo = match.groups()
                    repo_name_from_url = repo.replace('.git', '')
                    repo_to_supplier_map[repo_name_from_url] = owner
    except FileNotFoundError:
        print(f"⚠️ Warning: '{url_file_path}' not found. 'PackageSupplier' will default to 'Unknown'.")

    # --- メイン処理 ---
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
        print("No repositories found to run commands on.")

    for repo_name in repo_dirs:
        final_repo_dir = os.path.join(original_path, sbom_output_directory, repo_name)
        final_manifest_path = os.path.join(final_repo_dir, 'source', '_manifest')

        if os.path.isdir(final_manifest_path):
            print(f"🟢 Skipping: {repo_name} (SBOM manifest already exists)")
            print("-" * 50)
            continue

        repo_path = os.path.join(clone_to_directory, repo_name)
        print(f"▶️  Entering: {repo_name}")
        
        try:
            # --- パラメータ設定 ---
            package_name = repo_name
            # マップからサプライヤー情報を取得
            package_supplier = repo_to_supplier_map.get(repo_name, "Unknown")
            
            print(f"   Package Name: {package_name}")
            print(f"   Package Supplier: {package_supplier}")
            
            manifest_in_repo_path = os.path.join(repo_path, '_manifest')
            if os.path.isdir(manifest_in_repo_path):
                print("   Found existing '_manifest' directory. Removing it.")
                shutil.rmtree(manifest_in_repo_path)

            # --- sbom-tool実行---
            command = [
                'sbom-tool', 'generate', 
                '-bc', repo_path,
                '-b', repo_path,
                '-pn', package_name, 
                '-ps', package_supplier,
                '-m', repo_path,
                '-pv', 'latest'
            ]
            print(f"   Executing: {' '.join(command)}")
            subprocess.run(command, check=True, capture_output=True, text=True)
            print("✅ SBOM generation successful.")

            # --- 移動処理 ---
            source_manifest_path = manifest_in_repo_path
            
            if os.path.isdir(source_manifest_path):
                destination_dir = os.path.join(final_repo_dir, 'source')
                os.makedirs(destination_dir, exist_ok=True)
                
                print(f"   Moving '{source_manifest_path}' into: {destination_dir}")
                shutil.move(source_manifest_path, final_manifest_path)
                print("✅ Move successful.")
            else:
                print(f"⚠️ Warning: Could not find generated manifest directory at '{source_manifest_path}'")

        except FileNotFoundError:
            print(f"❌ Error: The command 'sbom-tool' was not found.")
            break
        except subprocess.CalledProcessError as e:
            print(f"❌ Error executing command in {repo_name}.")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

print("All processes finished.")

# Step 3: SBOMの生成 (Syft)

`Step 1`でクローンした各リポジトリに対し,**`syft`** を使用してSBOMを生成します.`syft`は,`requirements.txt`やGitHub Actionsのワークフローファイルなど,様々なパッケージマニフェストから依存関係を検出することに優れています.

### ## 事前準備 📝

* **`syft`** がシステムにインストールされ,PATHが通っている必要があります.
* Step 1が完了しており,**`cloned_repositories`** ディレクトリ内に分析対象のリポジトリが存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`cloned_repositories`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  **`syft` の実行**:
    * 各リポジトリのディレクトリに移動し,`syft dir:./ -o spdx-json`コマンドを実行します.
    * コマンドの標準出力をキャプチャし,リポジトリ内に`syft-sbom.json`という名前の一時ファイルとして保存します.

2.  **成果物の移動**:
    * 生成された`syft-sbom.json`を,後続の処理で扱いやすいように **`generated_sboms/<リポジトリ名>/source/`** ディレクトリ内に移動します.

すでに`syft-sbom.json`が最終的な保存先に存在する場合,そのリポジトリの処理はスキップされます.

In [ ]:
# --- 設定項目 ---
clone_to_directory = 'cloned_repositories'
sbom_output_directory = 'generated_sboms'

# --- 処理の開始 ---
print(f"--- Starting: Generating SBOMs with Syft ---")
print("-" * 50)

# 実行前のカレントディレクトリを保存する
original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    # クローンディレクトリ内のリポジトリ一覧を取得する
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
        print("No repositories found to run commands on.")

    for repo_name in repo_dirs:
        repo_path = os.path.join(clone_to_directory, repo_name)
        
        try:
            # --- 最終的なファイルパスを定義 ---
            destination_dir = os.path.join(original_path, sbom_output_directory, repo_name, 'source')
            final_sbom_filename = "syft-sbom.json"
            final_sbom_path = os.path.join(destination_dir, final_sbom_filename)

            # --- 処理をスキップするかの判定 ---
            # 既に成果物が存在する場合は処理をスキップする
            if os.path.exists(final_sbom_path):
                print(f"🟢 Skipping: {repo_name} (SBOM file already exists)")
                print("-" * 50)
                continue

            print(f"▶️  Entering: {repo_name}")
            os.chdir(repo_path)
            
            # --- syftコマンドの実行 ---
            temp_sbom_filename = 'syft-sbom.json'
            # 現在のディレクトリをスキャンし,SPDX JSON形式で出力する
            command = ['syft', 'dir:./', '-o', 'spdx-json']
            print(f"   Executing: {' '.join(command)} > {temp_sbom_filename}")
            
            result = subprocess.run(
                command, check=True, capture_output=True, text=True
            )
            print("✅ Syft execution successful.")

            # syftの標準出力を一時ファイルに書き込む
            with open(temp_sbom_filename, 'w', encoding='utf-8') as f:
                f.write(result.stdout)

            # --- 一時ファイルを最終的な場所に移動 ---
            os.makedirs(destination_dir, exist_ok=True)
            print(f"   Moving SBOM to: {final_sbom_path}")
            shutil.move(temp_sbom_filename, final_sbom_path)
            print("✅ SBOM file saved.")

        except FileNotFoundError:
            print(f"❌ Error: The command 'syft' or was not found.")
            break
        except subprocess.CalledProcessError as e:
            print(f"❌ Error executing command in {repo_name}.")
            if e.stdout:
                print(f"   [stdout]:\n{e.stdout.strip()}")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
        finally:
            # カレントディレクトリを元に戻す
            os.chdir(original_path)
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

print("All processes finished.")

# Step 4: SBOMの取得 (GitHub Dependency Graph API)

このステップでは,GitHubの **Dependency Graph API** を利用して,各リポジトリのSBOM（Software Bill of Materials）を取得します.このAPIから得られるSBOMは,特に**ライセンス**や**著作権情報**が豊富に含まれているという長所があります.

### ## 事前準備 📝

* **Dependency Graphの有効化**: 分析対象のリポジトリで,**Dependency Graph**機能が有効になっている必要があります.これは通常,パブリックリポジトリではデフォルトで有効ですが,プライベートリポジトリでは手動での有効化が必要です.（`Settings` > `Code security and analysis`）
* **`url_list.txt`**: Step 1で作成した,リポジトリのURLが記載されたファイルが必要です.

### ## 実行内容 ⚙️

以下のコードは,`url_list.txt`を読み込み,各リポジトリに対して以下の処理を自動的に実行します.

1.  **APIリクエスト**:
    * URLからリポジトリの所有者と名前を抽出し,適切なAPIエンドポイント（`api.github.com/repos/{owner}/{repo}/dependency-graph/sbom`）を構築します.
    * 構築したURLに対して`GET`リクエストを送信し,SBOMデータをJSON形式で取得します.

2.  **成果物の保存**:
    * APIから正常に取得できたSBOMデータを,**`generated_sboms/<リポジトリ名>/source/dependency-graph-sbom.json`** という名前のファイルとして保存します.

既に成果物ファイルが存在する場合,APIへのリクエストを節約するため,そのリポジトリの処理はスキップされます.

In [ ]:
# --- 設定項目 ---

# 1. GitHubリポジトリのURLリストが書かれたファイル
url_file_path = 'url_list.txt'

# 2. 生成されたSBOM(JSONファイル)を保存するディレクトリ
sbom_output_directory = 'generated_sboms'


# --- 処理の開始 ---
print("--- Starting: Fetching SBOMs from GitHub API (No Token) ---")
print("-" * 50)

original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    with open(url_file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print(f"❌ Error: '{url_file_path}' was not found.")
    urls = []

for repo_url in urls:
    # URLからownerとrepoを抽出
    match = re.search(r"github\.com/([^/]+)/([^/.]+)", repo_url)
    if not match:
        print(f"⚠️ Warning: Could not parse owner/repo from URL: {repo_url}")
        continue
    
    owner, repo_name = match.groups()
    
    # --- 保存先のパスを定義し,スキップ判定 ---
    destination_dir = os.path.join(original_path, sbom_output_directory, repo_name, 'source')
    final_sbom_path = os.path.join(destination_dir, 'dependency-graph-sbom.json')

    if os.path.exists(final_sbom_path):
        print(f"🟢 Skipping: {repo_name} (SBOM file already exists)")
        print("-" * 50)
        continue
        
    print(f"▶️  Fetching SBOM for: {owner}/{repo_name}")

    # --- GitHub APIへのリクエスト (認証ヘッダーなし) ---
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/dependency-graph/sbom"
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    try:
        response = requests.get(api_url, headers=headers)
        
        if response.status_code == 200:
            print("✅ API request successful.")
            sbom_data = response.json().get('sbom')
            if not sbom_data:
                print(f"❌ Error: 'sbom' key not found in the API response for {repo_name}.")
                continue

            # --- SBOMをファイルに保存 ---
            os.makedirs(destination_dir, exist_ok=True)
            print(f"   Writing SBOM to: {final_sbom_path}")
            
            with open(final_sbom_path, 'w', encoding='utf-8') as f:
                json.dump(sbom_data, f, ensure_ascii=False, indent=2)
            print("✅ SBOM file saved.")

        elif response.status_code == 404:
            print(f"⚠️ Warning: Could not fetch SBOM for {repo_name}. (Status: 404)")
            print("   The repository may not exist, or the Dependency Graph may not be enabled.")
        
        else:
            print(f"❌ Error: Failed to fetch SBOM for {repo_name}. Status code: {response.status_code}")
            print(f"   Response: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Error: A network error occurred while contacting the GitHub API.")
        print(f"   Details: {e}")
    
    finally:
        print("-" * 50)

print("All processes finished.")

# Step 5: SBOMの生成 (Trivy)

このステップでは,オープンソースの脆弱性スキャナである **`Trivy`** を使用して,各リポジトリからSBOMを生成します.`Trivy`は,ファイルシステム内のパッケージ依存関係を高速かつ広範囲に検出する能力に優れています.

### ## 事前準備 📝

* **`Trivy`** がシステムにインストールされ,PATHが通っている必要があります.
* Step 1が完了しており,**`cloned_repositories`** ディレクトリ内に分析対象のリポジトリが存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`cloned_repositories`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  **`Trivy` の実行**:
    * 各リポジトリのディレクトリに移動し,`trivy fs . --format spdx-json --output spdx-json-by-trivy.json` コマンドを実行します.
    * これにより,リポジトリ内に`spdx-json-by-trivy.json`という名前でSBOMファイルが直接生成されます.

2.  **成果物の移動とリネーム**:
    * 生成された`spdx-json-by-trivy.json`を,後続の処理で統一的に扱えるように **`generated_sboms/<リポジトリ名>/source/trivy-sbom.json`** という名前に変更して移動します.

すでに`trivy-sbom.json`が最終的な保存先に存在する場合,そのリポジトリの処理はスキップされます.

In [ ]:
# --- 設定項目 ---

# クローンされたリポジトリの保存ディレクトリ
clone_to_directory = 'cloned_repositories'
# 生成されたSBOMの保存ルートディレクトリ
sbom_output_directory = 'generated_sboms'

# --- 処理の開始 ---
# Trivyを使ったSBOM生成処理開始メッセージを表示
print(f"--- Starting: Generating SBOMs with Trivy ---")
print("-" * 50)

# 現在の作業ディレクトリを保存
original_path = os.getcwd()
# SBOM出力ディレクトリが存在しない場合は作成
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    # クローンディレクトリ内のリポジトリ一覧を取得
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
        print("No repositories found to run commands on.")

    # 各リポジトリに対してTrivyを使ったSBOM生成処理を実行
    for repo_name in repo_dirs:
        repo_path = os.path.join(clone_to_directory, repo_name)
        
        try:
            # --- 最終的なファイルパスを定義 ---
            # SBOMの最終的な保存先ディレクトリを構築
            destination_dir = os.path.join(original_path, sbom_output_directory, repo_name, 'source')
            
            # 生成されるSBOMのファイル名を 'trivy-sbom.json' に固定
            final_sbom_filename = "trivy-sbom.json"
            
            # 最終的なSBOMファイルのフルパスを構築
            final_sbom_path = os.path.join(destination_dir, final_sbom_filename)

            # --- 処理をスキップするかの判定 ---
            # 既にSBOMファイルが存在する場合はスキップ
            if os.path.exists(final_sbom_path):
                print(f"🟢 Skipping: {repo_name} (SBOM file already exists)")
                print("-" * 50)
                continue

            # --- ここからリポジトリ内での処理 ---
            print(f"▶️  Entering: {repo_name}")
            # カレントディレクトリをリポジトリのパスに変更
            os.chdir(repo_path)
            
            # --- trivyコマンドの実行 ---
            # trivyが出力する一時ファイル名を定義
            temp_sbom_filename = 'spdx-json-by-trivy.json'
            # trivyコマンドを構築
            command = ['trivy', 'fs', '.', '--format', 'spdx-json', '--output', temp_sbom_filename]
            print(f"   Executing: {' '.join(command)}")
            
            # trivyコマンドを実行
            # Trivyはファイルに直接出力するため,出力のキャプチャは不要
            subprocess.run(command, check=True, capture_output=True, text=True)
            print("✅ Trivy execution successful.")

            # --- 一時ファイルを最終的な場所に移動 ---
            # 最終的な保存先ディレクトリが存在しない場合は作成
            os.makedirs(destination_dir, exist_ok=True)
            print(f"   Moving SBOM to: {final_sbom_path}")
            # 一時ファイルを最終的な保存先にリネームしながら移動
            shutil.move(temp_sbom_filename, final_sbom_path)
            print("✅ SBOM file saved.")

        except FileNotFoundError:
            # 'trivy'コマンドが見つからない場合のエラー処理
            print(f"❌ Error: The command 'trivy' was not found.")
            break
        except subprocess.CalledProcessError as e:
            # コマンド実行中にエラーが発生した場合の処理
            print(f"❌ Error executing command in {repo_name}.")
            if e.stdout:
                print(f"   [stdout]:\n{e.stdout.strip()}")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
        finally:
            # カレントディレクトリを元のパスに戻す
            os.chdir(original_path)
            print("-" * 50)

except FileNotFoundError:
    # クローンディレクトリが見つからない場合のエラー処理
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

# 全てのTrivyを使ったSBOM生成処理が完了
print("All processes finished.")

# Step 6: ベースSBOMの作成と整形

このステップでは,`Step 2`で`sbom-tool`が出力した`manifest.spdx.json`をコピーし,後続の処理の基礎となる **`combined_sbom.json`** を作成します.同時に,JSONファイルのトップレベルのキーの順序を統一することで,ファイルの一貫性を保ちます.

### ## 事前準備 📝

* `Step 2`の`sbom-tool`によるSBOM生成が完了している必要があります.
* **`generated_sboms/<リポジトリ名>/source/_manifest`** ディレクトリ内に,`manifest.spdx.json`が存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`generated_sboms`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  `source/_manifest/spdx_2.2/manifest.spdx.json` を **`combined_sbom.json`** として各リポジトリのルートにコピーします.
2.  コピー直後に`combined_sbom.json`を再度開き,定義された`key_order`リストの順序に従ってキーを並び替えます.
3.  並び替えた内容でファイルを上書き保存します.

In [ ]:
# --- 設定項目 ---

# 1. 処理対象の親ディレクトリ
target_directory = 'generated_sboms'

# 2. 生成するファイル名
target_filename = 'combined_sbom.json'

# 3. 並び替えたいキーの順番
key_order = [
    "SPDXID",
    "spdxVersion",
    "creationInfo",
    "name",
    "dataLicense",
    "documentNamespace",
    "documentDescribes",
    "externalDocumentRefs",
    "packages",
    "files",
    "relationships",
]


# --- 処理の開始 ---
print(f"--- Starting: Copying and reordering SBOMs ---")
print("-" * 50)

try:
    # 'generated_sboms' 内のリポジトリ名を取得
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    
    if not repo_dirs:
         print(f"No repository directories found in '{target_directory}'.")

    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        # --- パスの定義 ---
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', '_manifest', 'spdx_2.2', 'manifest.spdx.json')
        destination_sbom_path = os.path.join(target_directory, repo_name, target_filename)

        # --- スキップ判定 ---
        if os.path.exists(destination_sbom_path):
            print(f"🟢 Skipping: '{target_filename}' already exists.")
            print("-" * 50)
            continue
        
        # --- コピー元の存在確認 ---
        if not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: sbom-tool SBOM not found for {repo_name}. Skipping.")
            print("-" * 50)
            continue

        try:
            # --- ステップ1: コピー ---
            print(f"   Copying sbom-tool's output to '{target_filename}'...")
            shutil.copy(source_sbom_path, destination_sbom_path)
            print("   ✅ Copy successful.")

            # --- ステップ2: 読み込みと並び替え ---
            with open(destination_sbom_path, 'r', encoding='utf-8') as f:
                original_data = json.load(f)

            ordered_data = {}
            # 指定された順でキーを追加
            for key in key_order:
                if key in original_data:
                    ordered_data[key] = original_data[key]
            # 残りのキーを末尾に追加
            for key, value in original_data.items():
                if key not in ordered_data:
                    ordered_data[key] = value

            # --- ステップ3: 整形して上書き保存 ---
            print(f"   Reordering keys...")
            with open(destination_sbom_path, 'w', encoding='utf-8') as f:
                json.dump(ordered_data, f, indent=2, ensure_ascii=False)
            print(f"   ✅ Keys reordered and saved.")

        except json.JSONDecodeError:
            print(f"❌ Error: Could not parse source JSON file. It may be corrupted.")
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The source directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 7: SBOMの統合 (GitHub Dependency Graph)

このステップでは,`Step 6`で作成したベースSBOM (`combined_sbom.json`) に対して,`Step 4`でGitHub APIから取得した`dependency-graph-sbom.json`の情報を統合します.この処理により,特に**ライセンス**や**著作権**に関する情報が大幅に補強されます.

### ## 実行内容 ⚙️

以下のコードは,2つのSBOMファイルを比較し,`combined_sbom.json`をより完全なものにするために,以下の4つの主要な処理を実行します.

#### 1. 既存パッケージ情報の補完
`purl`（Package URL）を基準に2つのSBOM間でパッケージを照合し,`combined_sbom.json`のパッケージ情報に不足があれば,`dependency-graph-sbom.json`から以下の情報を補完します.

* **`licenseConcluded`**: ライセンス情報が`NOASSERTION`の場合に更新します.
* **`copyrightText`**: 著作権情報が`NOASSERTION`の場合に更新します.

#### 2. 不足パッケージの追加
`sbom-tool`では検出されなかったが,`dependency-graph`では検出されたパッケージ（主にGitHub Actionsなど）を追加します.重複を防ぐため,**`name`（パッケージ名）**が`combined_sbom.json`に存在しないもののみが追加対象となります.

#### 3. `creators`情報の追記
`creationInfo`セクションに,`dependency-graph`を生成したツールの情報（`Tool: ...`）を追記します.

#### 4. ドキュメントコメントの追記
ファイル全体に関する`comment`フィールドに,`dependency-graph`からの注釈を追記します.

In [ ]:
def get_purl_from_package(pkg):
    """パッケージ情報からpurl（Package URL）を抽出する."""
    if 'externalRefs' in pkg:
        for ref in pkg['externalRefs']:
            if ref.get('referenceType') == 'purl':
                return ref.get('referenceLocator')
    return None

# --- 設定項目 ---
target_directory = 'generated_sboms'
base_sbom_filename = 'combined_sbom.json'
source_sbom_filename = 'dependency-graph-sbom.json'

# --- 処理の開始 ---
print(f"--- Starting: Supplementing '{base_sbom_filename}' with richer data and annotations ---")
print("-" * 50)

try:
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        base_sbom_path = os.path.join(target_directory, repo_name, base_sbom_filename)
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', source_sbom_filename)

        if not os.path.exists(base_sbom_path) or not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: One or both SBOM files are missing. Skipping.")
            print("-" * 50)
            continue

        try:
            with open(base_sbom_path, 'r', encoding='utf-8') as f:
                base_data = json.load(f)
            with open(source_sbom_path, 'r', encoding='utf-8') as f:
                source_data = json.load(f)

            # --- 変更項目をカウントするためのカウンターを初期化 ---
            # これにより、どの種類の情報がいくつ更新されたかを追跡する.
            licenses_updated = 0
            copyrights_updated = 0
            new_packages_added = 0
            new_relationships_added = 0
            creators_added = 0
            comments_updated = 0

            # --- ステップ2: 既存パッケージの情報を補完し、注釈を追加 ---
            # 補完元パッケージ情報をpurlをキーに辞書化し、後の検索処理を高速化する.
            source_package_map = {}
            if 'packages' in source_data:
                for pkg in source_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl:
                        source_package_map[purl] = {
                            'licenseConcluded': pkg.get('licenseConcluded', 'NOASSERTION'),
                            'copyrightText': pkg.get('copyrightText', 'NOASSERTION')
                        }

            # ベースSBOMの全パッケージをループし、情報が不足していれば補完する.
            if 'packages' in base_data:
                for pkg in base_data['packages']:
                    purl = get_purl_from_package(pkg)
                    # 同じpurlを持つパッケージが補完元にも存在する場合のみ処理を行う.
                    if purl and purl in source_package_map:
                        source_pkg_info = source_package_map[purl]
                        # このパッケージでどのフィールドを補完したかを記録するためのリスト.
                        supplemented_fields = []

                        # ライセンス情報が未表明の場合、dependency-graphの情報で補完する.
                        if pkg.get('licenseConcluded') == 'NOASSERTION' and source_pkg_info['licenseConcluded'] != 'NOASSERTION':
                            pkg['licenseConcluded'] = source_pkg_info['licenseConcluded']
                            supplemented_fields.append('licenseConcluded')
                            licenses_updated += 1
                        
                        # 著作権情報が未表明の場合、dependency-graphの情報で補完する.
                        if pkg.get('copyrightText') == 'NOASSERTION' and source_pkg_info['copyrightText'] != 'NOASSERTION':
                            pkg['copyrightText'] = source_pkg_info['copyrightText']
                            supplemented_fields.append('copyrightText')
                            copyrights_updated += 1
                        
                        # 1つ以上のフィールドが補完された場合、その出典をSPDXのannotationsとして記録する.
                        if supplemented_fields:
                            annotation = {
                                "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                                "annotationType": "OTHER",
                                "annotator": "Tool: sbom-merge-script",
                                "comment": f"Fields ({', '.join(supplemented_fields)}) were supplemented by dependency-graph."
                            }
                            pkg.setdefault('annotations', []).append(annotation)
            
            # --- ステップ3: 不足パッケージを追加し、注釈とRelationshipも追加 ---
            if 'packages' in source_data:
                # 親となるトップレベルパッケージのSPDXIDを特定する. これは新しい関係性を追加する際の親要素となる.
                main_package_spdx_id = None
                for rel in base_data.get('relationships', []):
                    if rel.get('relationshipType') == 'DESCRIBES' and rel.get('spdxElementId') == 'SPDXRef-DOCUMENT':
                        main_package_spdx_id = rel.get('relatedSpdxElement')
                        break
                
                # ベースSBOMに存在するパッケージ名をセットに格納し、重複チェックを高速化する.
                base_pkg_names = {pkg.get('name') for pkg in base_data.get('packages', []) if pkg.get('name')}
                
                for source_pkg in source_data.get('packages', []):
                    pkg_name = source_pkg.get('name')
                    # パッケージ名がベースSBOMに存在しない場合のみ追加処理を行う.
                    if pkg_name and pkg_name not in base_pkg_names:
                        # 追加するパッケージ自体に出典情報を注釈として記録する.
                        annotation = {
                            "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                            "annotationType": "OTHER",
                            "annotator": "Tool: sbom-merge-script",
                            "comment": "Package added from dependency-graph."
                        }
                        source_pkg.setdefault('annotations', []).append(annotation)
                        
                        # packagesリストに新しいパッケージを追加する.
                        base_data.setdefault('packages', []).append(source_pkg)
                        base_pkg_names.add(pkg_name)
                        new_packages_added += 1
                        
                        # 新しく追加したパッケージの親子関係をrelationshipsリストに追加し、SBOMの整合性を保つ.
                        new_pkg_spdx_id = source_pkg.get('SPDXID')
                        if main_package_spdx_id and new_pkg_spdx_id:
                            # ソースSBOMから関連するRelationship Typeを動的に探し出す.
                            found_relationship_type = 'CONTAINS'
                            for source_rel in source_data.get('relationships', []):
                                if source_rel.get('relatedSpdxElement') == new_pkg_spdx_id:
                                    found_relationship_type = source_rel.get('relationshipType', 'CONTAINS')
                                    break
                            
                            new_relationship = {
                                'spdxElementId': main_package_spdx_id,
                                'relatedSpdxElement': new_pkg_spdx_id,
                                'relationshipType': found_relationship_type,
                                'comment': 'Relationship added from dependency-graph.'
                            }
                            # 既存のrelationshipsに同じ関係性がなければ追加する.
                            if new_relationship not in base_data.get('relationships', []):
                                base_data.setdefault('relationships', []).append(new_relationship)
                                new_relationships_added += 1

            # --- ステップ4: creationInfo.creators の情報を追記 ---
            if 'creationInfo' in source_data and 'creators' in source_data['creationInfo'] and 'creationInfo' in base_data and 'creators' in base_data['creationInfo']:
                base_creators = base_data['creationInfo']['creators']
                for creator in source_data['creationInfo']['creators']:
                    if creator not in base_creators:
                        base_creators.append(creator)
                        creators_added += 1

            # --- ステップ5: ドキュメントコメントを追記 ---
            source_doc_comment = source_data.get('comment')
            if source_doc_comment:
                comment_header = "Note from dependency-graph"
                full_comment_to_add = f"{comment_header}: {source_doc_comment}"
                if 'comment' not in base_data or not base_data.get('comment'):
                    base_data['comment'] = full_comment_to_add
                    comments_updated = 1
                elif full_comment_to_add not in base_data['comment']:
                    base_data['comment'] += f"\\n\\n{full_comment_to_add}"
                    comments_updated = 1

            # --- ステップ6: 変更があった場合のみサマリーを表示して保存 ---
            total_changes = licenses_updated + copyrights_updated + new_packages_added + new_relationships_added + creators_added + comments_updated
            if total_changes > 0:
                print("\n   --- Summary of Changes ---")
                if licenses_updated > 0: print(f"   - Licenses updated: {licenses_updated}")
                if copyrights_updated > 0: print(f"   - Copyrights updated: {copyrights_updated}")
                if new_packages_added > 0: print(f"   - New packages added: {new_packages_added}")
                if new_relationships_added > 0: print(f"   - New relationships added: {new_relationships_added}")
                if creators_added > 0: print(f"   - Creators added: {creators_added}")
                if comments_updated > 0: print(f"   - Document comment updated: {comments_updated}")
                print(f"   --------------------------\n   Total changes: {total_changes}. Saving file...")
                
                # キーの順序を定義し、それに従ってファイルを再構築する.
                key_order = ["SPDXID", "spdxVersion", "creationInfo", "name", "dataLicense", "documentNamespace", "comment", "documentDescribes", "externalDocumentRefs", "packages", "files", "relationships"]
                ordered_data = {key: base_data[key] for key in key_order if key in base_data}
                ordered_data.update({key: value for key, value in base_data.items() if key not in ordered_data})
                with open(base_sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(ordered_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Successfully supplemented '{base_sbom_filename}'.")
            else:
                print("   No fields needed updating or adding.")

        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 8: SBOMの統合 (Syft)

`sbom-tool`とGitHub APIの情報を統合した`combined_sbom.json`に対し,さらに`syft`が検出した情報をマージしてSBOMを完成させます.`syft`は,パッケージの提供者（`supplier`）や作成元（`originator`）,検出場所（`sourceInfo`）,そして脆弱性スキャンに不可欠な**CPE**（Common Platform Enumeration）などの詳細情報を検出する能力に優れています.

### ## 実行内容 ⚙️

以下のコードは,2つのSBOMファイルを比較し,`combined_sbom.json`をさらにリッチなものにするために,以下の4つの主要な処理を実行します.

#### 1. 既存パッケージ情報の補完
`purl`を基準にパッケージを照合し,`combined_sbom.json`のパッケージ情報に不足があれば,`syft-sbom.json`から以下の情報を補完します.

* **`supplier`** と **`originator`**: `NOASSERTION`の場合に更新します.
* **`sourceInfo`**: 項目が存在しない場合に追加します.
* **`externalRefs`**: `purl`や`cpe`などの外部リンクが重複しないように追記します.

#### 2. 不足パッケージの追加
`sbom-tool`やGitHub APIでは検出されなかったが,`syft`では検出されたパッケージを追加します.重複を防ぐため,**`name`（パッケージ名）**が`combined_sbom.json`に存在しないもののみが追加対象となります.

#### 3. `creators`情報の追記
`creationInfo`セクションに,`syft`を生成したツールの情報（`Tool: syft-...`）を追記します.

#### 4. 整形して保存
最後に,すべての情報が統合された`combined_sbom.json`のキーの順序を統一的なフォーマットに整え,ファイルを上書き保存します.

In [ ]:
def get_purl_from_package(pkg):
    """パッケージ情報からpurl（Package URL）を抽出する."""
    if 'externalRefs' in pkg:
        for ref in pkg['externalRefs']:
            if ref.get('referenceType') == 'purl':
                return ref.get('referenceLocator')
    return None

# --- 設定項目 ---
target_directory = 'generated_sboms'
base_sbom_filename = 'combined_sbom.json'
source_sbom_filename = 'syft-sbom.json'

# --- 処理の開始 ---
print(f"--- Starting: Supplementing '{base_sbom_filename}' with data from syft and adding annotations ---")
print("-" * 50)

try:
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        base_sbom_path = os.path.join(target_directory, repo_name, base_sbom_filename)
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', source_sbom_filename)

        if not os.path.exists(base_sbom_path) or not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: One or both SBOM files are missing. Skipping.")
            print("-" * 50)
            continue

        try:
            with open(base_sbom_path, 'r', encoding='utf-8') as f:
                base_data = json.load(f)
            with open(source_sbom_path, 'r', encoding='utf-8') as f:
                source_data = json.load(f)

            # --- 変更項目をカウントするためのカウンターを初期化 ---
            # これにより、どの種類の情報がいくつ更新されたかを追跡する.
            suppliers_updated = 0
            originators_updated = 0
            source_infos_added = 0
            external_refs_added = 0
            new_packages_added = 0
            new_relationships_added = 0
            creators_added = 0

            # --- ステップ2: syftのパッケージ情報をpurlをキーにした辞書に整理 ---
            # 補完元パッケージ情報をpurlをキーに辞書化し、後の検索処理を高速化する.
            source_package_map = {}
            if 'packages' in source_data:
                for pkg in source_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl not in source_package_map:
                        source_package_map[purl] = pkg

            # --- ステップ3: 既存パッケージの情報を補完し、注釈を追加 ---
            # ベースSBOMの全パッケージをループし、情報が不足していれば補完する.
            if 'packages' in base_data:
                for pkg in base_data['packages']:
                    purl = get_purl_from_package(pkg)
                    # 同じpurlを持つパッケージが補完元にも存在する場合のみ処理を行う.
                    if purl and purl in source_package_map:
                        source_pkg = source_package_map[purl]
                        # このパッケージでどのフィールドを補完したかを記録するためのリスト.
                        supplemented_fields = []

                        # supplierが未表明の場合、syftの情報で補完する.
                        if (not pkg.get('supplier') or pkg.get('supplier') == 'NOASSERTION') and \
                           source_pkg.get('supplier') and source_pkg.get('supplier') != 'NOASSERTION':
                            pkg['supplier'] = source_pkg['supplier']
                            supplemented_fields.append('supplier')
                            suppliers_updated += 1
                        
                        # originatorが未表明の場合、syftの情報で補完する.
                        if (not pkg.get('originator') or pkg.get('originator') == 'NOASSERTION') and \
                           source_pkg.get('originator') and source_pkg.get('originator') != 'NOASSERTION':
                            pkg['originator'] = source_pkg['originator']
                            supplemented_fields.append('originator')
                            originators_updated += 1

                        # sourceInfoが存在しない場合、syftの情報を追加する.
                        if 'sourceInfo' not in pkg and source_pkg.get('sourceInfo'):
                            pkg['sourceInfo'] = source_pkg['sourceInfo']
                            supplemented_fields.append('sourceInfo')
                            source_infos_added += 1
                        
                        # externalRefs (CPEなど) に重複がないように情報を追記する.
                        refs_before = len(pkg.get('externalRefs', []))
                        if 'externalRefs' not in pkg: pkg['externalRefs'] = []
                        existing_locators = {ref.get('referenceLocator') for ref in pkg['externalRefs']}
                        for new_ref in source_pkg.get('externalRefs', []):
                            if new_ref.get('referenceLocator') not in existing_locators:
                                pkg['externalRefs'].append(new_ref)
                        refs_after = len(pkg['externalRefs'])
                        if refs_after > refs_before:
                            supplemented_fields.append('externalRefs')
                            external_refs_added += (refs_after - refs_before)

                        # 1つ以上のフィールドが補完された場合、その出典をSPDXのannotationsとして記録する.
                        if supplemented_fields:
                            annotation = {
                                "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                                "annotationType": "OTHER",
                                "annotator": "Tool: sbom-merge-script",
                                "comment": f"Fields ({', '.join(supplemented_fields)}) were supplemented by syft."
                            }
                            pkg.setdefault('annotations', []).append(annotation)

            # --- ステップ4: 不足パッケージを追加し、関連する全てのRelationshipも追加 ---
            if 'packages' in source_data:
                # 親となるトップレベルパッケージのSPDXIDを特定する.
                main_package_spdx_id = None
                for rel in base_data.get('relationships', []):
                    if rel.get('relationshipType') == 'DESCRIBES' and rel.get('spdxElementId') == 'SPDXRef-DOCUMENT':
                        main_package_spdx_id = rel.get('relatedSpdxElement')
                        break
                
                # ベースSBOMに存在するパッケージ名をセットに格納し、重複チェックを高速化する.
                base_pkg_names = {pkg.get('name') for pkg in base_data.get('packages', []) if pkg.get('name')}
                
                for source_pkg in source_data.get('packages', []):
                    pkg_name = source_pkg.get('name')
                    if pkg_name and pkg_name not in base_pkg_names:
                        # 追加するパッケージ自体に出典情報を注釈として記録する.
                        annotation = {
                            "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                            "annotationType": "OTHER",
                            "annotator": "Tool: sbom-merge-script",
                            "comment": "Package added from syft."
                        }
                        source_pkg.setdefault('annotations', []).append(annotation)
                        
                        base_data.setdefault('packages', []).append(source_pkg)
                        base_pkg_names.add(pkg_name)
                        new_packages_added += 1
                        
                        # 新しく追加したパッケージの親子関係をrelationshipsリストに追加し、SBOMの整合性を保つ.
                        new_pkg_spdx_id = source_pkg.get('SPDXID')
                        if main_package_spdx_id and new_pkg_spdx_id:
                            # ソースSBOMから関連する全てのRelationshipを探す (breakしない).
                            for source_rel in source_data.get('relationships', []):
                                if source_rel.get('relatedSpdxElement') == new_pkg_spdx_id:
                                    # 新しいRelationshipオブジェクトを作成する.
                                    new_relationship = {
                                        'spdxElementId': main_package_spdx_id,
                                        'relatedSpdxElement': new_pkg_spdx_id,
                                        'relationshipType': source_rel.get('relationshipType', 'CONTAINS'),
                                        'comment': 'Relationship added from syft.'
                                    }
                                    # 既存のrelationshipsに同じ関係性がなければ追加する (コメントは比較対象外).
                                    is_duplicate = any(
                                        rel.get('spdxElementId') == new_relationship['spdxElementId'] and
                                        rel.get('relatedSpdxElement') == new_relationship['relatedSpdxElement'] and
                                        rel.get('relationshipType') == new_relationship['relationshipType']
                                        for rel in base_data.get('relationships', [])
                                    )
                                    if not is_duplicate:
                                        base_data.setdefault('relationships', []).append(new_relationship)
                                        new_relationships_added += 1

            # --- ステップ5: syftのツール情報をcreatorsに追加 ---
            if 'creationInfo' in source_data and 'creators' in source_data['creationInfo']:
                base_creators = base_data.setdefault('creationInfo', {}).setdefault('creators', [])
                for creator in source_data['creationInfo']['creators']:
                    if creator not in base_creators:
                        base_creators.append(creator)
                        creators_added += 1
            
            # --- ステップ6: 変更があった場合のみサマリーを表示して保存 ---
            total_changes = (suppliers_updated + originators_updated + source_infos_added + 
                             external_refs_added + new_packages_added + 
                             new_relationships_added + creators_added)
            
            if total_changes > 0:
                print("\n   --- Summary of Changes (syft) ---")
                if suppliers_updated > 0: print(f"   - Suppliers updated: {suppliers_updated}")
                if originators_updated > 0: print(f"   - Originators updated: {originators_updated}")
                if source_infos_added > 0: print(f"   - Source infos added: {source_infos_added}")
                if external_refs_added > 0: print(f"   - External refs added: {external_refs_added}")
                if new_packages_added > 0: print(f"   - New packages added: {new_packages_added}")
                if new_relationships_added > 0: print(f"   - New relationships added: {new_relationships_added}")
                if creators_added > 0: print(f"   - Creators added: {creators_added}")
                print(f"   ----------------------------------\n   Total changes: {total_changes}. Saving file...")
                
                key_order = ["SPDXID", "spdxVersion", "creationInfo", "name", "dataLicense", "documentNamespace", "comment", "documentDescribes", "externalDocumentRefs", "packages", "files", "relationships"]
                ordered_data = {key: base_data[key] for key in key_order if key in base_data}
                ordered_data.update({key: value for key, value in base_data.items() if key not in ordered_data})
                with open(base_sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(ordered_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Successfully supplemented '{base_sbom_filename}'.")
            else:
                print("   No new information to supplement from syft.")

        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 9: SBOMの統合 (Trivy)

このステップでは,これまでの処理で情報を充実させてきた`combined_sbom.json`に対し,最後に`Trivy`が生成した`trivy-sbom.json`の情報を統合します.`Trivy`は,パッケージの目的 (`primaryPackagePurpose`) や,ツール固有の注釈 (`annotations`) といった,他のツールでは得られないユニークな情報を提供します.

### ## 実行内容 ⚙️

以下のコードは,`combined_sbom.json`を最終的に完成させるため,以下の4つの主要な処理を実行します.

#### 1. 既存パッケージ情報の補完
`purl`を基準にパッケージを照合し,`combined_sbom.json`のパッケージ情報に不足があれば,`trivy-sbom.json`から以下の情報を補完します.

* **`primaryPackagePurpose`**: パッケージの主な目的（例: `LIBRARY`, `APPLICATION`）が`NOASSERTION`または未設定の場合に更新します.
* **`annotations`**: `Trivy`による注釈情報（例: `PkgType: pip`）が未設定の場合に追加します.

#### 2. 不足パッケージの追加
これまでのツールでは検出されなかったが,`Trivy`が独自に検出したパッケージを追加します.重複を防ぐため,**`name`（パッケージ名）**が`combined_sbom.json`に存在しないもののみが追加対象となります.

#### 3. `creators`情報の追記
`creationInfo`セクションに,`Trivy`を生成したツールの情報（`Tool: trivy-...`）を追記します.

#### 4. 整形して保存
最後に,すべての情報が統合された`combined_sbom.json`のキーの順序を統一的なフォーマットに整え,ファイルを上書き保存します.

In [ ]:
def get_purl_from_package(pkg):
    """パッケージ情報からpurl（Package URL）を抽出する."""
    if 'externalRefs' in pkg:
        for ref in pkg['externalRefs']:
            if ref.get('referenceType') == 'purl':
                return ref.get('referenceLocator')
    return None

# --- 設定項目 ---
target_directory = 'generated_sboms'
base_sbom_filename = 'combined_sbom.json'
source_sbom_filename = 'trivy-sbom.json'

# --- 処理の開始 ---
print(f"--- Starting: Supplementing '{base_sbom_filename}' with data from Trivy and adding annotations ---")
print("-" * 50)

try:
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        base_sbom_path = os.path.join(target_directory, repo_name, base_sbom_filename)
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', source_sbom_filename)

        if not os.path.exists(base_sbom_path) or not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: One or both SBOM files are missing. Skipping.")
            print("-" * 50)
            continue

        try:
            with open(base_sbom_path, 'r', encoding='utf-8') as f:
                base_data = json.load(f)
            with open(source_sbom_path, 'r', encoding='utf-8') as f:
                source_data = json.load(f)

            # --- 変更項目をカウントするためのカウンターを初期化 ---
            purposes_updated = 0
            annotations_added = 0
            new_packages_added = 0
            new_relationships_added = 0
            creators_added = 0

            # --- ステップ2: Trivyのパッケージ情報をpurlをキーにした辞書に整理 ---
            # 補完元パッケージ情報をpurlをキーに辞書化し、後の検索処理を高速化する.
            source_package_map = {}
            if 'packages' in source_data:
                for pkg in source_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl not in source_package_map:
                        source_package_map[purl] = pkg

            # --- ステップ3: 既存パッケージの情報を補完し、注釈を追加 ---
            # ベースSBOMの全パッケージをループし、情報が不足していれば補完する.
            if 'packages' in base_data:
                for pkg in base_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl in source_package_map:
                        source_pkg = source_package_map[purl]
                        supplemented_fields = []
                        
                        # primaryPackagePurposeが未表明の場合、Trivyの情報で補完する.
                        if (not pkg.get('primaryPackagePurpose') or pkg.get('primaryPackagePurpose') == 'NOASSERTION') and \
                           source_pkg.get('primaryPackagePurpose') and source_pkg.get('primaryPackagePurpose') != 'NOASSERTION':
                            pkg['primaryPackagePurpose'] = source_pkg['primaryPackagePurpose']
                            supplemented_fields.append('primaryPackagePurpose')
                            purposes_updated += 1

                        # annotationsが存在しない場合、Trivyの情報を追加する.
                        if 'annotations' not in pkg and source_pkg.get('annotations'):
                            pkg['annotations'] = source_pkg['annotations']
                            supplemented_fields.append('annotations')
                            annotations_added += 1

                        # 1つ以上のフィールドが補完された場合、その出典をSPDXのannotationsとして記録する.
                        if supplemented_fields:
                            annotation = {
                                "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                                "annotationType": "OTHER",
                                "annotator": "Tool: sbom-merge-script",
                                "comment": f"Fields ({', '.join(supplemented_fields)}) were supplemented by trivy."
                            }
                            pkg.setdefault('annotations', []).append(annotation)

            # --- ステップ4: 不足パッケージを追加し、関連する全てのRelationshipも追加 ---
            if 'packages' in source_data:
                # 親となるトップレベルパッケージのSPDXIDを特定する.
                main_package_spdx_id = None
                for rel in base_data.get('relationships', []):
                    if rel.get('relationshipType') == 'DESCRIBES' and rel.get('spdxElementId') == 'SPDXRef-DOCUMENT':
                        main_package_spdx_id = rel.get('relatedSpdxElement')
                        break
                
                # ベースSBOMに存在するパッケージ名をセットに格納し、重複チェックを高速化する.
                base_pkg_names = {pkg.get('name') for pkg in base_data.get('packages', []) if pkg.get('name')}
                
                for source_pkg in source_data.get('packages', []):
                    pkg_name = source_pkg.get('name')
                    if pkg_name and pkg_name not in base_pkg_names:
                        # 追加するパッケージ自体に出典情報を注釈として記録する.
                        annotation = {
                            "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                            "annotationType": "OTHER",
                            "annotator": "Tool: sbom-merge-script",
                            "comment": "Package added from trivy."
                        }
                        source_pkg.setdefault('annotations', []).append(annotation)
                        
                        base_data.setdefault('packages', []).append(source_pkg)
                        base_pkg_names.add(pkg_name)
                        new_packages_added += 1
                        
                        # 新しく追加したパッケージの親子関係をrelationshipsリストに追加し、SBOMの整合性を保つ.
                        new_pkg_spdx_id = source_pkg.get('SPDXID')
                        if main_package_spdx_id and new_pkg_spdx_id:
                            # ソースSBOMから関連する全てのRelationshipを探す (breakしない).
                            for source_rel in source_data.get('relationships', []):
                                if source_rel.get('relatedSpdxElement') == new_pkg_spdx_id:
                                    new_relationship = {
                                        'spdxElementId': main_package_spdx_id,
                                        'relatedSpdxElement': new_pkg_spdx_id,
                                        'relationshipType': source_rel.get('relationshipType', 'CONTAINS'),
                                        'comment': 'Relationship added from trivy.'
                                    }
                                    # 既存のrelationshipsに同じ関係性がなければ追加する.
                                    is_duplicate = any(
                                        rel.get('spdxElementId') == new_relationship['spdxElementId'] and
                                        rel.get('relatedSpdxElement') == new_relationship['relatedSpdxElement'] and
                                        rel.get('relationshipType') == new_relationship['relationshipType']
                                        for rel in base_data.get('relationships', [])
                                    )
                                    if not is_duplicate:
                                        base_data.setdefault('relationships', []).append(new_relationship)
                                        new_relationships_added += 1

            # --- ステップ5: Trivyのツール情報をcreatorsに追加 ---
            if 'creationInfo' in source_data and 'creators' in source_data['creationInfo']:
                base_creators = base_data.setdefault('creationInfo', {}).setdefault('creators', [])
                for creator in source_data['creationInfo']['creators']:
                    if creator not in base_creators:
                        base_creators.append(creator)
                        creators_added += 1
            
            # --- ステップ6: 変更があった場合のみサマリーを表示して保存 ---
            total_changes = (purposes_updated + annotations_added + new_packages_added + 
                             new_relationships_added + creators_added)

            if total_changes > 0:
                print("\n   --- Summary of Changes (Trivy) ---")
                if purposes_updated > 0: print(f"   - Purposes updated: {purposes_updated}")
                if annotations_added > 0: print(f"   - Annotations added: {annotations_added}")
                if new_packages_added > 0: print(f"   - New packages added: {new_packages_added}")
                if new_relationships_added > 0: print(f"   - New relationships added: {new_relationships_added}")
                if creators_added > 0: print(f"   - Creators added: {creators_added}")
                print(f"   ---------------------------------\n   Total changes: {total_changes}. Saving file...")
                
                key_order = ["SPDXID", "spdxVersion", "creationInfo", "name", "dataLicense", "documentNamespace", "comment", "documentDescribes", "externalDocumentRefs", "packages", "files", "relationships"]
                ordered_data = {key: base_data[key] for key in key_order if key in base_data}
                ordered_data.update({key: value for key, value in base_data.items() if key not in ordered_data})
                with open(base_sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(ordered_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Successfully supplemented '{base_sbom_filename}'.")
            else:
                print("   No new information to supplement from Trivy.")

        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 10: `fileTypes`の追加によるSBOMの品質向上

これまでのステップで作成した`combined_sbom.json`をさらに改良し、SBOMとしての品質と実用性を高めます. この最終ステップでは、`files`セクションに含まれる各ファイルエントリに対し、そのファイルの役割を示す **`fileTypes`** フィールドを追加します.

### ## 実行内容 ⚙️

SPDXの仕様では、ファイルがソースコード、バイナリ、ドキュメントなど、どのカテゴリに属するかを示す`fileTypes`を定義できます. 以下のコードは、この仕様に基づき、各ファイルの拡張子を読み取って適切なタイプを自動で割り当てます.

1.  **`combined_sbom.json`の読み込み**:
    * 各リポジトリの`combined_sbom.json`を読み込み、`files`セクションを解析します.

2.  **ファイルタイプの判定**:
    * 事前に定義された拡張子のマッピング (`EXTENSION_TO_FILE_TYPE`) を基に、各ファイルの`fileName`からファイルタイプを判定します (例: `.py` → `SOURCE`, `.md` → `DOCUMENTATION`).
    * ファイル名が `.spdx.json` で終わる場合は、優先的に `SPDX` タイプとして分類します.
    * どのカテゴリにも一致しない拡張子は、汎用的な `OTHER` タイプとして分類されます.

3.  **`fileTypes`フィールドの追加**:
    * 判定したファイルタイプを、各ファイルエントリに`fileTypes`フィールドとして追加、または更新します.

In [ ]:
# --- 設定項目 ---
target_directory = 'generated_sboms'
target_filename = 'combined_sbom.json'

# --- 拡張子とSPDX FileTypeのマッピング ---
EXTENSION_TO_FILE_TYPE = {
    # SOURCE: 人間が読めるソースコード
    '.c': 'SOURCE', '.cpp': 'SOURCE', '.h': 'SOURCE', '.cs': 'SOURCE', 
    '.java': 'SOURCE', '.py': 'SOURCE', '.js': 'SOURCE', '.ts': 'SOURCE',
    '.go': 'SOURCE', '.rs': 'SOURCE', '.rb': 'SOURCE', '.sh': 'SOURCE', 
    '.html': 'SOURCE', '.css': 'SOURCE',

    # BINARY: コンパイルされたオブジェクトや実行可能ファイル
    '.o': 'BINARY', '.a': 'BINARY', '.exe': 'BINARY', '.dll': 'BINARY', 
    '.so': 'BINARY', '.bin': 'BINARY',

    # ARCHIVE: アーカイブファイル
    '.tar': 'ARCHIVE', '.jar': 'ARCHIVE', '.zip': 'ARCHIVE', '.gz': 'ARCHIVE',
    '.whl': 'ARCHIVE',

    # IMAGE: 画像ファイル
    '.jpg': 'IMAGE', '.jpeg': 'IMAGE', '.png': 'IMAGE', '.gif': 'IMAGE', 
    '.svg': 'IMAGE',

    # TEXT: 人間が読めるテキストファイル
    '.txt': 'TEXT',

    # AUDIO: オーディオファイル
    '.mp3': 'AUDIO', '.wav': 'AUDIO', '.ogg': 'AUDIO', '.flac': 'AUDIO',

    # VIDEO: ビデオファイル
    '.mp4': 'VIDEO', '.mov': 'VIDEO', '.avi': 'VIDEO', '.mkv': 'VIDEO',

    # DOCUMENTATION: ドキュメントとして機能するファイル
    '.md': 'DOCUMENTATION', '.rst': 'DOCUMENTATION', '.pdf': 'DOCUMENTATION',
    '.doc': 'DOCUMENTATION', '.docx': 'DOCUMENTATION',
}

# --- 処理の開始 ---
print(f"--- Starting: Adding 'fileTypes' and annotations to '{target_filename}' ---")
print("-" * 50)

try:
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")
        sbom_path = os.path.join(target_directory, repo_name, target_filename)

        if not os.path.exists(sbom_path):
            print(f"⚠️ Warning: '{target_filename}' not found. Skipping.")
            print("-" * 50)
            continue

        try:
            with open(sbom_path, 'r', encoding='utf-8') as f:
                sbom_data = json.load(f)

            if 'files' not in sbom_data or not sbom_data['files']:
                print("   No 'files' section found to update. Skipping.")
                print("-" * 50)
                continue

            changes_made = 0
            for file_entry in sbom_data['files']:
                filename = file_entry.get('fileName')
                if not filename:
                    continue

                if filename.lower().endswith('.spdx.json'):
                    file_type = 'SPDX'
                else:
                    _, extension = os.path.splitext(filename)
                    file_type = EXTENSION_TO_FILE_TYPE.get(extension.lower(), 'OTHER')
                
                # 'fileTypes' フィールドを追加または上書きする
                if file_entry.get('fileTypes') != [file_type]:
                    file_entry['fileTypes'] = [file_type]
                    changes_made += 1
                    
                    # 変更の出典を記録するための注釈を追加する
                    annotation = {
                        "annotationDate": datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
                        "annotationType": "OTHER",
                        "annotator": "Tool: file-type-script", # このスクリプト自身が注釈者
                        "comment": "Field (fileTypes) was added/updated based on file extension."
                    }
                    # 'annotations'リストがなければ作成し、注釈を追加する

            if changes_made > 0:
                with open(sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(sbom_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Added/Updated 'fileTypes' and annotations for {changes_made} files.")
            else:
                print("   All files already have correct 'fileTypes'. No changes needed.")

        except json.JSONDecodeError as e:
            print(f"❌ Error: Could not parse JSON file. Details: {e}")
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{target_directory}' was not found.")

print("All processes finished.")